## Restaurant Seat Optimization




#### Step 1: Import gurobipy module



In [11]:
import gurobipy as gp
from gurobipy import GRB
import random

#### Step 2: Define your model

In [12]:
m = gp.Model()

#### Step 3: Define your sets

In [13]:
#number of parties at a timeslot
I = 10

#number of timeslots
K = 7

#### Step 4: Define your parameters

In [20]:
#TODO: align on numbers for this, good candidate for sensitivity
# width and length of the dining room, in m
w = 1
l = 1


#initializing the party 
party_size = {}
for i in range(I):
    for k in range(K):
        party_size[i, k] = random.randint(2,6)
        #print(party_size[i,k])

# the table mapping below represents
# how many tables a specific party_size would require
table_mapping = { 
    2: 1,
    3: 2,
    4: 2,
    5: 3,
    6: 3
}

# initializing the amount of space that each of the party sizes 
# would take up
space = {}
for i in range(I):
    for k in range(K):
        num_tables = table_mapping.get(party_size[i, k])
        space[i, k] = 6 + 3 * num_tables

#### Step 5: Define your decision variables

In [21]:
# defining xik, whethere or not party i is seated at timeslot k
x = {}
for i in range(I):
    for k in range(K):
        x[i,k] = m.addVar(lb=0.0, obj=23.04*party_size[i,k], vtype=GRB.BINARY, name="x_"+str(i)+str(k))
        
#defining bk, whether or not there is a social distancing violation at timeslot k
b={}
for k in range(K):
    b[k] = m.addVar(lb=0.0, obj=-880, vtype=GRB.BINARY, name="b_"+str(k))
    
# defining the piecewise function for revenue
# model.setPWLObj()
s = {}
#defining space slack variable
for k in range(K):
    s[k] = m.addVar(lb=0.0, vtype=GRB.INTEGER, name="s_"+str(k))

#defining space excess variable
e = {}
for k in range(K):
    e[k] = m.addVar(lb=0.0, vtype=GRB.INTEGER, name="e_" + str(k))

# #defining C
# m.addVar(obj=-25000, vtype=GRB.BINARY, name="C") 

#### Step 6: Set your objective function

In [22]:
m.modelSense = GRB.MAXIMIZE

#### Step 7: Add Constraints

In [38]:
# defining space constraint
for k in range(K):
    running_sum = 0
    for i in range(I):
        #print(running_sum)
        xik = m.getVarByName("x_"+str(i) + str(k)).X
        print(xik)
#         spaceik = space[i,k]
#         running_sum += index_sum.getValue()
    print(running_sum)
    m.addConstr((s[k] - e[k] + running_sum), GRB.EQUAL, l*w)

print(f"l={l}, w={w}")
    
# defining the constraint that turns on the social distancing violated binary var
for k in range(K):
    m.addConstr(10**6*(1 - b[k]) + e[k], GRB.GREATER_EQUAL, 1)

# defining the constraint that makes sure that if there are social distancing violations,
# they take up at most 20% of the dining room
for k in range(K):
    m.addConstr(e[k], GRB.LESS_EQUAL, l * w * 0.2)
    
# defining the constraint that makes sure that each party only occupies one table
for k in range(K):
   # running_sum = 0;
    for i in range(I):
        #running_sum += x[i,k]
        m.addConstr(x[i,k], GRB.LESS_EQUAL, 1)

# for k in range(K):
#     m.addConstr(s[k], GRB.LESS_EQUAL, l*w*0.2)
    
m.update()

AttributeError: Unable to retrieve attribute 'X'

#### Step 8: Solve the model

In [24]:
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 182 rows, 182 columns and 350 nonzeros
Model fingerprint: 0x22520153
Variable types: 0 continuous, 182 integer (154 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [5e+01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+06]

MIP start from previous solve produced solution with objective 6474.24 (0.01s)
Loaded MIP start from previous solve with objective 6474.24

Presolve removed 182 rows and 182 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 6474.24 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.474240000000e+03, best bound 6.474240000000e+03, gap 0.0000%


#### Step 8: Print variable values  (The Messy Way)

In [39]:
print("Printing result\n")
print('\nProfit: %g' % m.objVal) #gets the objective function value
print('\nSOLUTION')
for myVars in m.getVars():
    print(myVars)
    print('%s %g' % (myVars.varName, myVars.x))

Printing result



AttributeError: Unable to retrieve attribute 'objVal'

In [10]:
m.write("checkModel.lp")